**1. Data analyze and cleaning**

Loading data, clearing text from noise, lemmatization

In [2]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import spacy
from tqdm import tqdm

In [3]:
df=pd.read_csv('data.csv')
display(df.head(5))
print(df.shape)
print(df['relevant'].value_counts())

,text,relevant
0,"Единственный в стране, нисколько не сомневаюсь...",1
1,Замечательная женщина и преподаватель!🌺,1
2,"Пересдача,курсовая ,удача 😁не все так плохо в...",1
3,Для потенциальных абитуриентов (студентов) пол...,1
4,"Рейтинг Вуза это конечно здорово, но 5 бюджетн...",1


(43553, 2)
relevant
1    31177
0    12376
Name: count, dtype: int64


In [ ]:
# Stop-words
russian_stopwords = set(stopwords.words("russian"))

# Lemmatizator
nlp = spacy.load("ru_core_news_sm")

# Регулярные выражения
RE_HTML = re.compile(r'<.*?>')
RE_EMOJI = re.compile(r'[\U00010000-\U0001FFFF]', flags=re.UNICODE)

# Clean funtion
def clean_text(text):
    text = str(text).lower() # Lowercase
    text = RE_HTML.sub('', text) # HTML
    text = RE_EMOJI.sub('', text) # Emoji
    text = re.sub(r'[^\w\s]', ' ', text) # Punctuation marks
    text = re.sub(r'\d+', '', text) # Numbers
    text = re.sub(r'\s+', ' ', text).strip() # Extra spaces
    
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc]
    tokens = [t for t in tokens if t and t not in russian_stopwords] # if t remove empty tokens
    return ' '.join(tokens)


In [ ]:
# Cleaning
tqdm.pandas()
df['clean'] = df['text'].progress_apply(clean_text)
display(df.head(5))

100%|██████████| 43553/43553 [07:13<00:00, 100.44it/s]


,text,relevant,clean
0,"Единственный в стране, нисколько не сомневаюсь...",1,единственный страна нисколько сомневаться это ...
1,Замечательная женщина и преподаватель!🌺,1,замечательный женщина преподаватель
2,"Пересдача,курсовая ,удача 😁не все так плохо в...",1,пересдача курсовой удача плохой вроде
3,Для потенциальных абитуриентов (студентов) пол...,1,потенциальный абитуриент студент получить весь...
4,"Рейтинг Вуза это конечно здорово, но 5 бюджетн...",1,рейтинг вуз это здоровый бюджетный место посту...


In [ ]:
# Check
print(df['clean'].isna().sum())

0


In [ ]:
# Save
df.to_csv("clean_comments.csv", index=False)